## 6.3. 填充和步幅

卷积的输出形状$(n_h-k_h+1)\times(n_w-k_w+1)$取决于输入形状$n_h \times n_w$和卷积核的形状$k_h \times k_w$

还有什么因素会影响输出的大小呢？
- 填充 padding
- 步幅 stride

**为了与torch.nn的Conv2d中参数padding大小一致，这里我个人选择统一按照Conv2d的参数书写即p为单边填充数**

### 6.3.1. 填充

随着我们应用许多连续卷积层，累积丢失的像素数就多了。解决方案：**填充 padding**，在输入图像的边界填充元素（通常填充元素是$0$）
行填充$2*p_h$（一半顶一半底），列填充$2*p_w$（一半左一半右），输出形状为：$$(n_h-k_h+2*p_h+1) \times (n_w-k_w+2*p_w+1)$$，输出高度和宽度分别增加$2*p_h$和$2*p_w$。

一般情况下，$2*p_h = k_h-1, \ 2*p_w=k_w-1$

**注：原文中这里的$p$代表的是顶+底，或左+右的填充数之和，而在torch.nn中的Conv2d中的padding参数代表是单个边的填充个数，即padding * 2 = p。个人笔记这里已经统一为conv2d中的padding大小。**

卷积神经网络中**卷积核的高度和宽度通常为奇数**，如1、3、5、7。好处是，**保持空间维度**+**可以在上下左右填充数量对应相同**

当满足以下条件：
 1. 卷积核的大小是奇数；
 2. 所有边的填充行数和列数相同；
 3. 输出与输入具有相同高度和宽度

则可以得出：输出$Y[i, j]$是通过以输入$X[i, j]$为中心，与卷积核进行互相关计算得到的。


In [2]:
import torch
from torch import nn

def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

# 每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8,8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

torch.Size([8, 8])

In [3]:
# 使用高度为5，宽度为3的卷积核，高度和宽度两边的填充分别为2和1
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 6.3.2. 步幅

有时候为了高效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素

每次滑动元素的数量称为**步幅 stride**

垂直步幅为$s_h$，水平步幅为$s_w$时，输出形状：

$$[(n_h - k_h + 2*p_h + s_h)/s_h] \times [(n_w - k_w + 2*p_w + s_w)/s_w]$$

当$2*p = k - 1$，则简化为：

$$[(n_h + s_h - 1)/s_h] \times [(n_w + s_w - 1)/s_w]$$

In [4]:
# 将高度和宽度的步幅设置为2，从而将输入的高度和宽度减半
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

一个有点复杂的例子，k=(3, 5), p=(0, 1), s=(3, 4)

带入公式计算得到的n = (8/3, 9/4) = (2, 2)

其实实践中很少使用到这些例子

In [6]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

### 6.3.3 小结

填充可以增加输出的高度和宽度。这常用来使输出与输入具有相同的高和宽。

步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的$1/n$（$n$是一个大于$1$的整数）。

**填充和步幅可用于有效地调整数据的维度**。